# Olist E-commerce EDA

This notebook loads the Olist datasets and prepares cleaned versions for analysis.


In [2]:
!pip install pandas

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 24.0 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/11.1 MB 24.0 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 13.0 MB/s  0:00:00
   ---------------------------------------- 0.0/12.4 MB ? eta -:--:--
   ------------- -------------------------- 4.2/12.4 MB 20.2 MB/s eta 0:00:01
   ------------------------------ --------- 9.4/12.4 MB 22.4 MB/s eta 0:00:01
   ---------------------------------------- 12.4/12.4 MB 21.8 MB/s  0:00:00

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- -----


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 1) Setup and imports
import os
import glob
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)


In [7]:
# 2) Load data 
DATA_DIR = r"C:\Users\dbswl\OneDrive\바탕 화면\project\olist_eda\data"

def load_csv(filename):
    return pd.read_csv(os.path.join(DATA_DIR, filename))

orders = load_csv("olist_orders_dataset.csv")
customers = load_csv("olist_customers_dataset.csv")
order_items = load_csv("olist_order_items_dataset.csv")
payments = load_csv("olist_order_payments_dataset.csv")
reviews = load_csv("olist_order_reviews_dataset.csv")
products = load_csv("olist_products_dataset.csv")
sellers = load_csv("olist_sellers_dataset.csv")
geolocation = load_csv("olist_geolocation_dataset.csv")

print("orders", orders.shape)
print("customers", customers.shape)
print("order_items", order_items.shape)
print("payments", payments.shape)
print("reviews", reviews.shape)
print("products", products.shape)
print("sellers", sellers.shape)
print("geolocation", geolocation.shape)


orders (99441, 8)
customers (99441, 5)
order_items (112650, 7)
payments (103886, 5)
reviews (99224, 7)
products (32951, 9)
sellers (3095, 4)
geolocation (1000163, 5)


In [8]:

# 3) Review cleanup (simple version)
# 주문 id, 리뷰 점수, 리뷰 제목, 리뷰 내용 까지 같은 것 삭제 
reviews_dedup = reviews.copy()

# Make sure timestamps are proper datetime
reviews_dedup['review_answer_timestamp'] = pd.to_datetime(
    reviews_dedup['review_answer_timestamp'], errors='coerce'
)

# Sort so the latest review stays when duplicates exist
reviews_dedup = reviews_dedup.sort_values('review_answer_timestamp')

# Remove exact duplicates by order + content + score
reviews_dedup = reviews_dedup.drop_duplicates(
    subset=['order_id','review_score','review_comment_title','review_comment_message'],
    keep='last'
)

print("reviews before:", len(reviews))
print("reviews after dedup:", len(reviews_dedup))


reviews before: 99224
reviews after dedup: 98997


In [9]:
# 4) Order date cleanup (simple version)
# 승인일이 배송 인계일 보다 늦은지 표시
# 배송사 인계일이 배송완료일보다 늦은지 표시
# 위의 데이터들을 모두 nat 처리 
# 각 단계별 소요 시간을 시간 단위로 계산
orders_fix = orders.copy()

# Parse dates
orders_fix['order_purchase_timestamp'] = pd.to_datetime(orders_fix['order_purchase_timestamp'], errors='coerce')
orders_fix['order_approved_at'] = pd.to_datetime(orders_fix['order_approved_at'], errors='coerce')
orders_fix['order_delivered_carrier_date'] = pd.to_datetime(orders_fix['order_delivered_carrier_date'], errors='coerce')
orders_fix['order_delivered_customer_date'] = pd.to_datetime(orders_fix['order_delivered_customer_date'], errors='coerce')
orders_fix['order_estimated_delivery_date'] = pd.to_datetime(orders_fix['order_estimated_delivery_date'], errors='coerce')

# Fix impossible sequences (tolerate <= 3 day inversions)
approved_to_carrier_days = (
    orders_fix['order_delivered_carrier_date'] - orders_fix['order_approved_at']
).dt.total_seconds() / 86400
carrier_to_delivered_days = (
    orders_fix['order_delivered_customer_date'] - orders_fix['order_delivered_carrier_date']
).dt.total_seconds() / 86400

orders_fix.loc[approved_to_carrier_days < -3, 'order_delivered_carrier_date'] = pd.NaT
orders_fix.loc[carrier_to_delivered_days < -3, 'order_delivered_customer_date'] = pd.NaT


# Lead times (hours)
orders_fix['lead_purchase_to_approved_hr'] = (
    orders_fix['order_approved_at'] - orders_fix['order_purchase_timestamp']
).dt.total_seconds() / 3600

orders_fix['lead_approved_to_carrier_hr'] = (
    orders_fix['order_delivered_carrier_date'] - orders_fix['order_approved_at']
).dt.total_seconds() / 3600

orders_fix['lead_carrier_to_delivered_hr'] = (
    orders_fix['order_delivered_customer_date'] - orders_fix['order_delivered_carrier_date']
).dt.total_seconds() / 3600

orders_fix['lead_purchase_to_delivered_hr'] = (
    orders_fix['order_delivered_customer_date'] - orders_fix['order_purchase_timestamp']
).dt.total_seconds() / 3600

orders_fix['delay_days'] = (
    orders_fix['order_delivered_customer_date'] - orders_fix['order_estimated_delivery_date']
).dt.days


In [20]:
# 5) Product cleanup (simple version)
# 카테고리 이름이 없는 것 = 같은 셀러 + 같은 무게 + 같은 치수 인 것 같은 것으로 대체!

products_fix = products.copy()

# ----------------------------
# 5-1) 기본 정리
# ----------------------------
products_fix['product_category_name'] = products_fix['product_category_name'].fillna('Unknown')
products_fix['product_category_name'] = products_fix['product_category_name'].astype(str).str.strip()
products_fix.loc[products_fix['product_category_name'] == '', 'product_category_name'] = 'Unknown'

products_fix.loc[products_fix['product_weight_g'] <= 0, 'product_weight_g'] = pd.NA

print("Unknown category (before inference):",
      (products_fix['product_category_name'] == 'Unknown').sum())
print("weight NaN:", products_fix['product_weight_g'].isna().sum())

# ----------------------------
# 5-2) 카테고리 추론
# Rule: same seller + exact weight/size match -> assign most common category
# ----------------------------
products_fix['product_category_name_inferred'] = products_fix['product_category_name']

# Map product -> seller (mode seller per product)
prod_seller = order_items.groupby('product_id')['seller_id'].agg(
    lambda s: s.mode().iloc[0] if not s.mode().empty else pd.NA
)
products_fix['seller_id'] = products_fix['product_id'].map(prod_seller)

# Ensure numeric dims
dim_cols = [
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
]
for c in dim_cols:
    if c in products_fix.columns:
        products_fix[c] = pd.to_numeric(products_fix[c], errors='coerce')

# Build exact size/weight signature
sig = products_fix[dim_cols].astype('float').round(3)
products_fix['size_weight_sig'] = sig.astype(str).agg('|'.join, axis=1)

# Build lookup table from known categories only
known = products_fix[products_fix['product_category_name_inferred'] != 'Unknown']

lookup = (
    known
    .groupby(['seller_id', 'size_weight_sig'])['product_category_name_inferred']
    .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else pd.NA)
)

# Apply inference
mask_unknown = products_fix['product_category_name_inferred'] == 'Unknown'
products_fix.loc[mask_unknown, 'product_category_name_inferred'] = (
    products_fix.loc[mask_unknown]
    .set_index(['seller_id', 'size_weight_sig'])
    .index
    .map(lookup)
    .fillna('Unknown')
).values

print("Unknown after seller+size/weight inference:",
      (products_fix['product_category_name_inferred'] == 'Unknown').sum())

# Apply inferred category
products_fix['product_category_name'] = products_fix['product_category_name_inferred']

# Drop helper columns from inference
products_fix = products_fix.drop(
    columns=['product_category_name_inferred', 'seller_id', 'size_weight_sig'],
    errors='ignore'
)

# ----------------------------
# 5-3) ❗ 결측치 / Unknown 카테고리 제거 (최종 정리)
# ----------------------------
before_cnt = len(products_fix)

products_fix = products_fix[
    products_fix['product_category_name'].notna() &
    (products_fix['product_category_name'] != 'Unknown')
].copy()

after_cnt = len(products_fix)

print("Products before drop:", before_cnt)
print("Products after drop (category known only):", after_cnt)
print("Dropped products:", before_cnt - after_cnt)


Unknown category (before inference): 610
weight NaN: 6
Unknown after seller+size/weight inference: 498
Products before drop: 32951
Products after drop (category known only): 32453
Dropped products: 498


In [21]:
# 5.1) Industry Classification Mapping
# Mapping Portuguese categories to standardized Industry Groups and Types
INDUSTRY_MAP = {
    'cama_mesa_banho': ('Home_Living', 'Bed_Bath'),
    'moveis_decoracao': ('Home_Living', 'Furniture'),
    'moveis_escritorio': ('Home_Living', 'Furniture'),
    'moveis_sala': ('Home_Living', 'Furniture'),
    'utilidades_domesticas': ('Home_Living', 'Kitchen_Housewares'),
    'casa_construcao': ('Home_Living', 'Home_Care'),
    'ferramentas_jardim': ('Home_Living', 'Home_Care'),
    'informatica_acessorios': ('Tech_Electronics', 'Computers_Telephony'),
    'pcs': ('Tech_Electronics', 'Computers_Telephony'),
    'telefonia': ('Tech_Electronics', 'Computers_Telephony'),
    'eletrodomesticos': ('Tech_Electronics', 'Appliances'),
    'eletroportateis': ('Tech_Electronics', 'Appliances'),
    'climatizacao': ('Tech_Electronics', 'Appliances'),
    'eletronicos': ('Tech_Electronics', 'Entertainment'),
    'consoles_games': ('Tech_Electronics', 'Entertainment'),
    'perfumaria': ('Health_Beauty', 'Beauty'),
    'beleza_saude': ('Health_Beauty', 'Beauty'),
    'esporte_lazer': ('Sports_Leisure', 'Sports_Leisure'),
    'instrumentos_musicais': ('Sports_Leisure', 'Music_Art'),
    'artes': ('Sports_Leisure', 'Music_Art'),
    'relogios_presentes': ('Fashion_Accessories', 'Accessories_Gifts'),
    'malas_acessorios': ('Fashion_Accessories', 'Accessories_Gifts'),
    'fashion_bolsas_e_acessorios': ('Fashion_Accessories', 'Accessories_Gifts'),
    'fashion_calcados': ('Fashion_Accessories', 'Clothing_Shoes'),
    'bebes': ('Kids_Toys', 'Kids_Toys'),
    'brinquedos': ('Kids_Toys', 'Kids_Toys'),
    'automotivo': ('Automotive', 'Automotive'),
    'papelaria': ('Others', 'Books_Stationery'),
    'livros_interesse_geral': ('Others', 'Books_Stationery'),
    'agro_industria_e_comercio': ('Others', 'Industry_Others'),
    'pet_shop': ('Others', 'Industry_Others'),
    'construcao_ferramentas_seguranca': ('Others', 'Industry_Others'),
    'sinalizacao_e_seguranca': ('Others', 'Industry_Others'),
    'cool_stuff': ('Others', 'Industry_Others')
}

def classify_industry(category_name):
    # Returns (Industry_Group, Industry_Type) for a given category name.
    return INDUSTRY_MAP.get(category_name, ('Others', 'Others'))

# Apply classification to products_fix
industry_info = products_fix['product_category_name'].apply(lambda x: pd.Series(classify_industry(x)))
products_fix[['industry_group', 'industry_type']] = industry_info

print("✅ Industry classification applied successfully!")
print("Industry Groups:", products_fix['industry_group'].unique().tolist())


✅ Industry classification applied successfully!
Industry Groups: ['Health_Beauty', 'Sports_Leisure', 'Kids_Toys', 'Home_Living', 'Others', 'Tech_Electronics', 'Fashion_Accessories', 'Automotive']


In [22]:
# 6) Payment cleanup (simple version)
# 바우처로 사서 공짜로 받은건 남기고 undefined만 지운다. 

payments_fix = payments.copy()

# Drop clearly invalid zero-value rows (not_defined only)
payments_fix = payments_fix[~((payments_fix['payment_value'] == 0) & (payments_fix['payment_type'] == 'not_defined'))].copy()

payments_pos = payments_fix[payments_fix['payment_value'] > 0].copy()

print("payments total:", len(payments_fix))
print("payments <= 0:", (payments_fix['payment_value'] <= 0).sum())
print("payments_pos:", len(payments_pos))


payments total: 103883
payments <= 0: 6
payments_pos: 103877


In [23]:
# 7) Geolocation cleanup (simple version)
# Aggregate by zip prefix and keep representative city/state.
# 위경도 평균값 사용 
# 도시 주 최빈값 사용
# 이걸 왜 이렇게 했냐? 굳이 정확한 위치기 우리한테 당장 필요 없다고 생각해서 같은 주인거 다 묶어버림
geo = geolocation.copy()

geolocation_fix = geo.groupby('geolocation_zip_code_prefix').agg(
    geolocation_lat=('geolocation_lat','mean'),
    geolocation_lng=('geolocation_lng','mean'),
    geolocation_city=('geolocation_city', lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]),
    geolocation_state=('geolocation_state', lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0]),
    count=('geolocation_city','size')
).reset_index()

print("geolocation_fix:", geolocation_fix.shape)


geolocation_fix: (19015, 6)


In [24]:
# 8) Payment vs item totals check (simple version)
# Count orders with large discrepancies (abs delta > 100).
## 주문별 주문 합계 + 배송비 합계 가 페이먼트와 얼만큼 차이나는지 
## 100 이상 차이가 나는 주문들은 체크한다.(2~3만원)차이  -> 할부로 인한 것으로 추정 됨 
items_tot = order_items.groupby('order_id').agg(
    items_total=('price','sum'),
    freight_total=('freight_value','sum')
)
items_tot['items_plus_freight'] = items_tot['items_total'] + items_tot['freight_total']

pay_tot = payments.groupby('order_id').agg(payment_total=('payment_value','sum'))

compare = items_tot.join(pay_tot, how='inner')
compare['delta'] = compare['payment_total'] - compare['items_plus_freight']

large_delta = compare[compare['delta'].abs() > 100]
print("large deltas (abs>100):", len(large_delta))


large deltas (abs>100): 3


In [25]:
# 9) Final cleaned dataset bundle (fixed only)
clean = {
    'orders': orders_fix,
    'payments': payments_fix,
    'reviews': reviews_dedup,
    'products': products_fix,
    'geolocation': geolocation_fix,
}

print('orders = orders_fix', orders_fix.shape)
print('payments = payments_fix', payments_fix.shape)
print('reviews = reviews_dedup', reviews_dedup.shape)
print('products = products_fix', products_fix.shape)
print('geolocation = geolocation_fix', geolocation_fix.shape)


orders = orders_fix (99441, 13)
payments = payments_fix (103883, 5)
reviews = reviews_dedup (98997, 7)
products = products_fix (32453, 11)
geolocation = geolocation_fix (19015, 6)


In [27]:
products_fix

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,industry_group,industry_type
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,Health_Beauty,Beauty
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,Sports_Leisure,Music_Art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,Sports_Leisure,Sports_Leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,Kids_Toys,Kids_Toys
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,Home_Living,Kitchen_Housewares
...,...,...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0,Home_Living,Furniture
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0,Others,Others
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0,Home_Living,Bed_Bath
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0,Tech_Electronics,Computers_Telephony


In [28]:
import pandas as pd
import numpy as np

# ----------------------------
# 0) 입력 테이블 가져오기 (둘 중 하나 방식 지원)
# ----------------------------
def _get_table(name):
    # name: 'orders', 'order_items', 'products', 'payments'
    if 'clean' in globals() and isinstance(clean, dict):
        mapping = {
            'orders': 'orders',
            'order_items': 'order_items',   # clean에 없으면 전역에서 찾음
            'products': 'products',
            'payments': 'payments'
        }
        key = mapping.get(name, name)
        if key in clean:
            return clean[key]
    # fallback: 전역 변수에서 찾기
    if name == 'orders' and 'orders_fix' in globals(): return orders_fix
    if name == 'order_items' and 'order_items' in globals(): return order_items
    if name == 'products' and 'products_fix' in globals(): return products_fix
    if name == 'payments' and 'payments_fix' in globals(): return payments_fix
    raise KeyError(f"테이블을 찾을 수 없음: {name}. clean dict 또는 전역변수(orders_fix 등)를 확인해줘.")

orders_df = _get_table('orders').copy()
items_df  = _get_table('order_items').copy()
prod_df   = _get_table('products').copy()

# ----------------------------
# 1) 분석 설정
# ----------------------------
# 어떤 카테고리 기준으로 볼지 선택:
# - product_category_name (원본/추론 적용된 카테고리)
# - industry_group / industry_type (네가 만든 산업 분류)
CATEGORY_COL = 'industry_group'   # <- 필요하면 'industry_group' 또는 'industry_type'로 변경

# 매출(구매액) 계산 방식:
# - item_price_only: price 합 (보수적)
# - item_plus_freight: price + freight_value 합 (주문 금액에 가까움)
AMOUNT_MODE = 'item_plus_freight'  # 'item_price_only' or 'item_plus_freight'

# 어떤 주문 상태를 포함할지 (예: delivered만 보면 실제 구매완료 기준)
# 원본 olist_order_status 예: 'delivered', 'shipped', 'canceled', 'unavailable', ...
USE_STATUS_FILTER = True
ALLOWED_STATUS = {'delivered'}  # 필요시 {'delivered','shipped'} 등으로 수정

# 월 기준 날짜: purchase timestamp 기준
DATE_COL = 'order_purchase_timestamp'

# ----------------------------
# 2) 기본 전처리(날짜/필터) + 조인(orders + items + products)
# ----------------------------
# 날짜 파싱 (이미 했어도 안전하게)
orders_df[DATE_COL] = pd.to_datetime(orders_df[DATE_COL], errors='coerce')

# 상태 필터
if USE_STATUS_FILTER and 'order_status' in orders_df.columns:
    orders_df = orders_df[orders_df['order_status'].isin(ALLOWED_STATUS)].copy()

# 월 컬럼 생성
orders_df['order_month'] = orders_df[DATE_COL].dt.to_period('M').dt.to_timestamp()

# items에 금액 컬럼 생성
items_df['price'] = pd.to_numeric(items_df['price'], errors='coerce')
items_df['freight_value'] = pd.to_numeric(items_df.get('freight_value', 0), errors='coerce')

if AMOUNT_MODE == 'item_price_only':
    items_df['item_amount'] = items_df['price']
elif AMOUNT_MODE == 'item_plus_freight':
    items_df['item_amount'] = items_df['price'] + items_df['freight_value']
else:
    raise ValueError("AMOUNT_MODE는 'item_price_only' 또는 'item_plus_freight'만 가능")

# 조인: items -> orders (월/고객) / items -> products (카테고리)
# orders의 customer_id가 필요하면 같이 가져옴
need_cols = ['order_id', 'order_month']
if 'customer_id' in orders_df.columns:
    need_cols.append('customer_id')

items_enriched = (
    items_df
    .merge(orders_df[need_cols], on='order_id', how='inner')
    .merge(prod_df[['product_id', CATEGORY_COL]], on='product_id', how='left')
)

# 카테고리 결측 처리
items_enriched[CATEGORY_COL] = items_enriched[CATEGORY_COL].fillna('Unknown').astype(str).str.strip()
items_enriched.loc[items_enriched[CATEGORY_COL] == '', CATEGORY_COL] = 'Unknown'

# 구매총량(수량) 정의:
# Olist order_items는 한 행이 한 아이템 라인(수량=1로 간주)이라 일반적으로 row count가 구매수량 역할.
items_enriched['item_qty'] = 1

# ----------------------------
# 3) 월별 카테고리 KPI 집계
# ----------------------------
agg_dict = {
    'item_qty': ('item_qty', 'sum'),
    'purchase_amount': ('item_amount', 'sum'),
    'orders': ('order_id', 'nunique'),
}
if 'customer_id' in items_enriched.columns:
    agg_dict['buyers'] = ('customer_id', 'nunique')

monthly_cat = (
    items_enriched
    .groupby(['order_month', CATEGORY_COL], as_index=False)
    .agg(**agg_dict)
    .sort_values(['order_month', CATEGORY_COL])
)

# 파생 KPI (AOV 등)
monthly_cat['amount_per_order'] = monthly_cat['purchase_amount'] / monthly_cat['orders'].replace(0, np.nan)
monthly_cat['items_per_order']  = monthly_cat['item_qty'] / monthly_cat['orders'].replace(0, np.nan)

# ----------------------------
# 4) 성장지표(MoM, 3M rolling MoM, YoY)
# ----------------------------
def add_growth_metrics(df, group_col, value_cols, date_col='order_month'):
    df = df.sort_values([group_col, date_col]).copy()
    for col in value_cols:
        # MoM 성장률
        df[f'{col}_mom'] = df.groupby(group_col)[col].pct_change()

        # 3개월 이동평균 기반 MoM (노이즈 줄인 성장률)
        roll = df.groupby(group_col)[col].rolling(3, min_periods=2).mean().reset_index(level=0, drop=True)
        df[f'{col}_3m_ma'] = roll
        df[f'{col}_3m_ma_mom'] = df.groupby(group_col)[f'{col}_3m_ma'].pct_change()

        # YoY 성장률 (12개월 전 대비) - 데이터가 충분할 때 의미 있음
        df[f'{col}_yoy'] = df.groupby(group_col)[col].pct_change(12)
    return df

monthly_cat = add_growth_metrics(
    monthly_cat,
    group_col=CATEGORY_COL,
    value_cols=['item_qty', 'purchase_amount', 'orders'] + (['buyers'] if 'buyers' in monthly_cat.columns else [])
)

# ----------------------------
# 5) (옵션) 카테고리 Top-N만 보기 / 피벗 테이블 만들기
# ----------------------------
# 최근 3개월 구매액 기준 Top 10 카테고리
recent_month = monthly_cat['order_month'].max()
last3 = monthly_cat[monthly_cat['order_month'] >= (recent_month - pd.DateOffset(months=2))]
top10 = (
    last3.groupby(CATEGORY_COL)['purchase_amount'].sum()
    .sort_values(ascending=False)
    .head(10)
    .index
)
monthly_cat_top10 = monthly_cat[monthly_cat[CATEGORY_COL].isin(top10)].copy()

# 구매액 피벗(월 x 카테고리) - 시각화/보고서용
pivot_amount = monthly_cat.pivot_table(
    index='order_month', columns=CATEGORY_COL, values='purchase_amount', aggfunc='sum'
).sort_index()

pivot_qty = monthly_cat.pivot_table(
    index='order_month', columns=CATEGORY_COL, values='item_qty', aggfunc='sum'
).sort_index()

# ----------------------------
# 6) 결과 저장 (원하면)
# ----------------------------
# monthly_cat.to_csv("monthly_category_kpis.csv", index=False, encoding="utf-8-sig")
# pivot_amount.to_csv("pivot_monthly_amount.csv", encoding="utf-8-sig")
# pivot_qty.to_csv("pivot_monthly_qty.csv", encoding="utf-8-sig")

print("monthly_cat:", monthly_cat.shape)
display(monthly_cat.head(20))


monthly_cat: (191, 24)


C:\Users\dbswl\AppData\Local\Temp\ipykernel_45672\3922621806.py:129: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f'{col}_3m_ma_mom'] = df.groupby(group_col)[f'{col}_3m_ma'].pct_change()
C:\Users\dbswl\AppData\Local\Temp\ipykernel_45672\3922621806.py:129: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f'{col}_3m_ma_mom'] = df.groupby(group_col)[f'{col}_3m_ma'].pct_change()
C:\Users\dbswl\AppData\Local\Temp\ipykernel_45672\3922621806.py:129: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Eit

,order_month,industry_group,item_qty,purchase_amount,orders,buyers,amount_per_order,items_per_order,item_qty_mom,item_qty_3m_ma,item_qty_3m_ma_mom,item_qty_yoy,purchase_amount_mom,purchase_amount_3m_ma,purchase_amount_3m_ma_mom,purchase_amount_yoy,orders_mom,orders_3m_ma,orders_3m_ma_mom,orders_yoy,buyers_mom,buyers_3m_ma,buyers_3m_ma_mom,buyers_yoy
1,2016-10-01,Automotive,8,1351.27,8,8,168.908750,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2017-01-01,Automotive,31,5386.38,27,27,199.495556,1.148148,2.875000,19.500000,NaN,NaN,2.986161,3368.825000,NaN,NaN,2.375000,17.500000,NaN,NaN,2.375000,17.500000,NaN,NaN
20,2017-02-01,Automotive,83,14301.63,75,75,190.688400,1.106667,1.677419,40.666667,1.085470,NaN,1.655147,7013.093333,1.081762,NaN,1.777778,36.666667,1.095238,NaN,1.777778,36.666667,1.095238,NaN
29,2017-03-01,Automotive,78,15304.25,67,67,228.421642,1.164179,-0.060241,64.000000,0.573770,NaN,0.070105,11664.086667,0.663187,NaN,-0.106667,56.333333,0.536364,NaN,-0.106667,56.333333,0.536364,NaN
38,2017-04-01,Automotive,86,17409.92,79,79,220.378734,1.088608,0.102564,82.333333,0.286458,NaN,0.137587,15671.933333,0.343606,NaN,0.179104,73.666667,0.307692,NaN,0.179104,73.666667,0.307692,NaN
47,2017-05-01,Automotive,114,20273.15,100,100,202.731500,1.140000,0.325581,92.666667,0.125506,NaN,0.164460,17662.440000,0.127011,NaN,0.265823,82.000000,0.113122,NaN,0.265823,82.000000,0.113122,NaN
56,2017-06-01,Automotive,110,31615.42,101,101,313.023960,1.089109,-0.035088,103.333333,0.115108,NaN,0.559473,23099.496667,0.307832,NaN,0.010000,93.333333,0.138211,NaN,0.010000,93.333333,0.138211,NaN
65,2017-07-01,Automotive,114,16192.75,103,103,157.211165,1.106796,0.036364,112.666667,0.090323,NaN,-0.487821,22693.773333,-0.017564,NaN,0.019802,101.333333,0.085714,NaN,0.019802,101.333333,0.085714,NaN
74,2017-08-01,Automotive,161,22303.65,128,128,174.247266,1.257812,0.412281,128.333333,0.139053,NaN,0.377385,23370.606667,0.029825,NaN,0.242718,110.666667,0.092105,NaN,0.242718,110.666667,0.092105,NaN
83,2017-09-01,Automotive,107,16486.08,96,96,171.730000,1.114583,-0.335404,127.333333,-0.007792,NaN,-0.260835,18327.493333,-0.215789,NaN,-0.250000,109.000000,-0.015060,NaN,-0.250000,109.000000,-0.015060,NaN
